# CommaSeparatedListOutputParser
`CommaSeparatedListOutputParser`: 쉼표로 구분된 항목 목록을 반활할 필요가 있을때 List 형태로 변환하는 parser이다.

csv 파일 확장자들은 콤마로 구분된 파일이다. 이것을 콤마로 구분된 list형식으로 변환해준다.

이것은 후처리에 용이하다. 몇번째 인덱스를 가져오는지 등등!

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_teddynote import logging
logging.langsmith("CH03-OutputParser")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH03-OutputParser


In [3]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 콤마로 구분된 리스트 출력 파서 초기화
output_parser = CommaSeparatedListOutputParser()
# 지시사항을 불러올 수 있다.
format_instructions = output_parser.get_format_instructions()

In [4]:
print(format_instructions)

Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`


In [5]:
# template의 변수를 반드시 채워줘야한다. input_variables은 나중에 채우겠다.
prompt = PromptTemplate(
    template = "List five {subject}. \n{format_instructions}",
    input_variables=["subject"], # 입력 변수로 subject사용 - input_variables: 나중에 채우겠다. 여기선 invoke할때 채운다.
    # 부분 변수로 형식 지침 사용
    partial_variables = {"format_instructions": format_instructions},
)

model = ChatOpenAI(model='gpt-4o-mini')

chain = prompt | model | output_parser

In [6]:
answer = chain.invoke({"subject": "대한민국 관광명소"})
answer

['경복궁', '제주도', '부산 해운대', 'N서울타워', '전주 한옥마을']

In [7]:
type(answer)

list

In [11]:
chain.invoke("대한민국 관광명소")

['경복궁', 'N서울타워', '제주도', '부여 백제문화단지', '경주 불국사']

In [8]:
# list가 하나씩 나오기 때문에 원칙상 안쓰는게 맞다.
for s in chain.stream("대한민국 관광명소"):
    print(s)

['경복궁']
['남산서울타워']
['제주도']
['부산 해운대']
['경주 불국사']


In [9]:
#from langchain.output_parsers import StructuredOutputParser